<a href="https://colab.research.google.com/github/podemosaprender/xw1/blob/main/usd_uva.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Dolar vs. Inflacion


### (instalar software, etc.)

In [1]:
!pip install --upgrade plotly >/dev/null 2>&1 ;
;

''

In [2]:
PrepareForWeb= False
PrepareForWeb= True #U: descomentar esta línea antes de guardar en github

In [3]:
import pandas as pd #U: leer y agrupar datos
import numpy as np

import plotly.graph_objects as go #U: graficar con plotly
import plotly.express as px #U: graficar con plotly
import plotly #SEE: https://medium.com/@282abhishek/using-plotly-with-nbconvert-in-google-colab-96834c4f2850
if PrepareForWeb:
  plotly.offline.init_notebook_mode(connected=True)
else:
  plotly.io.renderers.default = 'colab'

#### Dólar vs Inflación en Argentina

In [5]:
uva_df_tables22= pd.read_html('http://data.podemosaprender.org/finanzas/UVA_BCRA_2022.html')
uva_df22= uva_df_tables22[0] #A: sabemos que hay una sola!
uva_df_tables23= pd.read_html('http://data.podemosaprender.org/finanzas/UVA_BCRA_2023.html')
uva_df23= uva_df_tables23[0] #A: sabemos que hay una sola!
uva_df= pd.concat([uva_df22, uva_df23])


uva_df.rename(columns={'Valor':'UVA', 'Fecha': 'fecha'}, inplace=True)
uva_df.fecha= pd.to_datetime( uva_df.fecha, format='%Y-%m-%d')

usd_blue_df22= pd.read_csv("https://data.podemosaprender.org/finanzas/USD_BLUE_2022.csv", sep="\t")
usd_blue_df23= pd.read_csv("https://data.podemosaprender.org/finanzas/USD_BLUE_2023.csv", sep="\t")
usd_blue_df= pd.concat([usd_blue_df22, usd_blue_df23])
usd_blue_df.fecha= pd.to_datetime( usd_blue_df.fecha, format='%Y/%m/%d')

uva_usd_df= pd.merge(uva_df, usd_blue_df, on="fecha")
uva_usd_df.rename(columns={'venta':'USD_v', 'compra': 'USD_c'}, inplace=True)
uva_usd_df.head()

,fecha,UVA,USD_c,USD_v
0,2022-01-03,97.58,202.0,202.0
1,2022-01-04,97.82,202.5,202.5
2,2022-01-05,97.89,203.5,203.5
3,2022-01-06,97.97,204.5,204.5
4,2022-01-07,98.05,204.0,204.0


In [6]:
fig = px.line(uva_usd_df, x="fecha", y=["UVA","USD_v"], title='Dólar Blue vs Inflación')
fig.update_layout(legend=dict(yanchor="top",y=0.99,xanchor="left",x=0.02))
fig.show()

In [7]:
uva_usd_df['usd_en_uva']= uva_usd_df.USD_v / uva_usd_df.UVA
uva_usd_df.head()

,fecha,UVA,USD_c,USD_v,usd_en_uva
0,2022-01-03,97.58,202.0,202.0,2.070096
1,2022-01-04,97.82,202.5,202.5,2.070129
2,2022-01-05,97.89,203.5,203.5,2.078864
3,2022-01-06,97.97,204.5,204.5,2.087374
4,2022-01-07,98.05,204.0,204.0,2.080571


In [9]:
fig = px.line(uva_usd_df, x="fecha", y=["usd_en_uva"], title='USD Blue/UVA (CPI)')
fig.update_layout(showlegend=False)
fig.show()

In [10]:
uva_usd_df['chg']= uva_usd_df.usd_en_uva.pct_change()

In [11]:
cnt_sube= uva_usd_df[uva_usd_df.chg>0].chg.count()
cnt_baja= uva_usd_df[uva_usd_df.chg<0].chg.count()
f"Sube {cnt_sube} dias vs Baja {cnt_baja} dias"

'Sube 216 dias vs Baja 277 dias'

In [12]:
counts, bins = np.histogram(uva_usd_df.chg, bins=np.arange(-0.15, 0.15, 0.01))
bins = 0.5 * (bins[:-1] + bins[1:])

fig = px.bar(x=bins, y=counts, labels={'x':'total_bill', 'y':'count'})
fig.show()